## Gradient Vanishing in RNN

eg. 
* "Cloud in the" => prediction: "sky"
* "I grew up in France. I speak fluent" => prediction: "French"


Present state is determined by not only the word right before but also words before a few word.

RNN predicts present state using past state. Thus it should have all past state information for accurate prediction.

This leads size of sequence gets longer, and might cause critical problem of 'DNN': 'Gradient Vanishing'

![LSTM1.png](LSTM1.png)

# LSTM: Long Short Term Memory
* is the concept made to solve gradient vanishing issue of RNN
* is model transformed hidden layer $h_t$ in RNN to $C_t$ in model below.


$$h_t = f(h_{t-1},x_t) = tanh(W_{hh} h_{t-1} + W_{xh} x_t)$$

<br>

$$C_t = f_t \cdot C_{t-1} + i_t \cdot h_t$$


![LSTM2.png](LSTM2.png)

* Determine importance of past and present through forget gate and input gate.
* Forget gate indicates how much forget past state and input gate indicates how much accept present state.

e.g.
```python
tensorflow.keras.layers.LSTM(
    units, activation='tanh', recurrent_activation='sigmoid',
    use_bias=True, kernel_initializer='glorot_uniform',
    recurrent_initializer='orthogonal',
    bias_initializer='zeros', unit_forget_bias=True,
    kernel_regularizer=None, recurrent_regularizer=None, bias_regularizer=None,
    activity_regularizer=None, kernel_constraint=None, recurrent_constraint=None,
    bias_constraint=None, dropout=0.0, recurrent_dropout=0.0,
    return_sequences=False, return_state=False, go_backwards=False, stateful=False,
    time_major=False, unroll=False, **kwargs
)
```
from [tensorflow.org](https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM)

* It is not proper to learn entire article as one sequence, when learning many sentences.
* In view of the entire article, preceeding and following sentences are less related.
* Implement batch learning
#####  Batch Learning
* is a method of learning by breaking it into independently recognized units. (e.g. 10 letters)
* Batch Learning decreases accuracy of model.

## Stacked RNN
* Enlarge the number of cells to increase the accuracy of the model.
* Use a Stacked RNN that spans multiple recurrent layers.

![StackedRNN.png](StackedRNN.png)

In [2]:
import numpy as np
import urllib.request
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

lines = ["if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea."]

text = ' '.join(lines)
print("Text: ",text)
print('Length of string (number of letters): %d' % len(text))

Text:  if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
Length of string (number of letters): 180


In [3]:
# Set eliminates duplicates.
char_vocab = sorted(list(set(text)))
vocab_size=len(char_vocab)
print ('Size of letter set: {}'.format(vocab_size))

# char_to_index using enumerate
char_to_index = dict((c, i) for i, c in enumerate(char_vocab)) # 글자에 고유한 정수 인덱스 부여
print("char_to_index:",char_to_index)

# index_to_char using char_to_index
index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print("index_to_char:",index_to_char)

Size of letter set: 25
char_to_index: {' ': 0, "'": 1, ',': 2, '.': 3, 'a': 4, 'b': 5, 'c': 6, 'd': 7, 'e': 8, 'f': 9, 'g': 10, 'h': 11, 'i': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'r': 19, 's': 20, 't': 21, 'u': 22, 'w': 23, 'y': 24}
index_to_char: {0: ' ', 1: "'", 2: ',', 3: '.', 4: 'a', 5: 'b', 6: 'c', 7: 'd', 8: 'e', 9: 'f', 10: 'g', 11: 'h', 12: 'i', 13: 'k', 14: 'l', 15: 'm', 16: 'n', 17: 'o', 18: 'p', 19: 'r', 20: 's', 21: 't', 22: 'u', 23: 'w', 24: 'y'}


In [5]:
# Sampling in 10 letters unit
# Sequence length=10; extract unit size =10
# Total number of samples: Divide the string into 10 equal parts.

seq_length = 10
n_samples = int(np.floor((len(text) - 1) / seq_length))
print ('문장 샘플의 수 : {}'.format(n_samples))

# Sampling
_tmp = []
for i in range(n_samples): 
    X_sample = text[i * seq_length: (i + 1) * seq_length]
    # 0:10 -> 10:20 -> 20:30로 loop를 돌면서 문장 샘플을 1개씩 가져온다.
    X_encoded = [char_to_index[c] for c in X_sample] # 하나의 문장 샘플에 대해서 정수 인코딩
    _tmp.append(X_encoded)

# Predict next unit(10 letters) using preceeding unit
train_X = _tmp[:-1]
train_y = _tmp[1:]
print("train_X:",train_X)
print("train_y:",train_y)

문장 샘플의 수 : 17
train_X: [[12, 9, 0, 24, 17, 22, 0, 23, 4, 16], [21, 0, 21, 17, 0, 5, 22, 12, 14, 7], [0, 4, 0, 20, 11, 12, 18, 2, 0, 7], [17, 16, 1, 21, 0, 7, 19, 22, 15, 0], [22, 18, 0, 18, 8, 17, 18, 14, 8, 0], [21, 17, 10, 8, 21, 11, 8, 19, 0, 21], [17, 0, 6, 17, 14, 14, 8, 6, 21, 0], [23, 17, 17, 7, 0, 4, 16, 7, 0, 7], [17, 16, 1, 21, 0, 4, 20, 20, 12, 10], [16, 0, 21, 11, 8, 15, 0, 21, 4, 20], [13, 20, 0, 4, 16, 7, 0, 23, 17, 19], [13, 2, 0, 5, 22, 21, 0, 19, 4, 21], [11, 8, 19, 0, 21, 8, 4, 6, 11, 0], [21, 11, 8, 15, 0, 21, 17, 0, 14, 17], [16, 10, 0, 9, 17, 19, 0, 21, 11, 8], [0, 8, 16, 7, 14, 8, 20, 20, 0, 12]]
train_y: [[21, 0, 21, 17, 0, 5, 22, 12, 14, 7], [0, 4, 0, 20, 11, 12, 18, 2, 0, 7], [17, 16, 1, 21, 0, 7, 19, 22, 15, 0], [22, 18, 0, 18, 8, 17, 18, 14, 8, 0], [21, 17, 10, 8, 21, 11, 8, 19, 0, 21], [17, 0, 6, 17, 14, 14, 8, 6, 21, 0], [23, 17, 17, 7, 0, 4, 16, 7, 0, 7], [17, 16, 1, 21, 0, 4, 20, 20, 12, 10], [16, 0, 21, 11, 8, 15, 0, 21, 4, 20], [13, 20, 0, 4, 16, 7, 0, 

In [6]:
# One-hot Encoding
encoded_X = to_categorical(train_X)
encoded_y = to_categorical(train_y)
print('encoded_X.shape : {}'.format(encoded_X.shape))
print('encoded_y.shape : {}'.format(encoded_y.shape))

encoded_X.shape : (16, 10, 25)
encoded_y.shape : (16, 10, 25)


In [7]:
n_hidden = 50
model = Sequential()
model.add(LSTM(n_hidden, input_shape=(None, encoded_X.shape[2]), return_sequences=True))
model.add(LSTM(n_hidden, return_sequences=True))
model.add(LSTM(n_hidden, return_sequences=True))
model.add(TimeDistributed(Dense(vocab_size, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(encoded_X, encoded_y, epochs=1000, verbose=1)

2021-07-22 13:39:11.377333: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-07-22 13:39:11.377373: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-07-22 13:39:11.377408: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fennecfox38-13UD580-GX30K): /proc/driver/nvidia/version does not exist
2021-07-22 13:39:11.377750: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-22 13:39:12.869233: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None 

Epoch 1/1000
1/1 [==============================] - 5s 5s/step - loss: 3.2174 - accuracy: 0.0812
Epoch 2/1000
1/1 [==============================] - 0s 35ms/step - loss: 3.2140 - accuracy: 0.1875
Epoch 3/1000
1/1 [==============================] - 0s 31ms/step - loss: 3.2104 - accuracy: 0.1875
Epoch 4/1000
1/1 [==============================] - 0s 22ms/step - loss: 3.2065 - accuracy: 0.1875
Epoch 5/1000
1/1 [==============================] - 0s 28ms/step - loss: 3.2022 - accuracy: 0.1875
Epoch 6/1000
1/1 [==============================] - 0s 29ms/step - loss: 3.1974 - accuracy: 0.1875
Epoch 7/1000
1/1 [==============================] - 0s 25ms/step - loss: 3.1921 - accuracy: 0.1875
Epoch 8/1000
1/1 [==============================] - 0s 27ms/step - loss: 3.1860 - accuracy: 0.1875
Epoch 9/1000
1/1 [==============================] - 0s 30ms/step - loss: 3.1790 - accuracy: 0.1875
Epoch 10/1000
1/1 [==============================] - 0s 29ms/step - loss: 3.1710 - accuracy: 0.1875
Epoch 11/10

1/1 [==============================] - 0s 22ms/step - loss: 2.6937 - accuracy: 0.1937
Epoch 84/1000
1/1 [==============================] - 0s 28ms/step - loss: 2.6862 - accuracy: 0.1937
Epoch 85/1000
1/1 [==============================] - 0s 19ms/step - loss: 2.6788 - accuracy: 0.1937
Epoch 86/1000
1/1 [==============================] - 0s 21ms/step - loss: 2.6712 - accuracy: 0.2000
Epoch 87/1000
1/1 [==============================] - 0s 25ms/step - loss: 2.6637 - accuracy: 0.2062
Epoch 88/1000
1/1 [==============================] - 0s 20ms/step - loss: 2.6563 - accuracy: 0.2000
Epoch 89/1000
1/1 [==============================] - 0s 23ms/step - loss: 2.6489 - accuracy: 0.2062
Epoch 90/1000
1/1 [==============================] - 0s 27ms/step - loss: 2.6411 - accuracy: 0.2188
Epoch 91/1000
1/1 [==============================] - 0s 26ms/step - loss: 2.6330 - accuracy: 0.2125
Epoch 92/1000
1/1 [==============================] - 0s 29ms/step - loss: 2.6244 - accuracy: 0.2125
Epoch 93/1000


1/1 [==============================] - 0s 32ms/step - loss: 1.9812 - accuracy: 0.3750
Epoch 165/1000
1/1 [==============================] - 0s 42ms/step - loss: 1.9731 - accuracy: 0.3875
Epoch 166/1000
1/1 [==============================] - 0s 20ms/step - loss: 1.9661 - accuracy: 0.3812
Epoch 167/1000
1/1 [==============================] - 0s 18ms/step - loss: 1.9595 - accuracy: 0.3812
Epoch 168/1000
1/1 [==============================] - 0s 25ms/step - loss: 1.9522 - accuracy: 0.3875
Epoch 169/1000
1/1 [==============================] - 0s 22ms/step - loss: 1.9444 - accuracy: 0.3812
Epoch 170/1000
1/1 [==============================] - 0s 25ms/step - loss: 1.9369 - accuracy: 0.3875
Epoch 171/1000
1/1 [==============================] - 0s 25ms/step - loss: 1.9306 - accuracy: 0.3812
Epoch 172/1000
1/1 [==============================] - 0s 17ms/step - loss: 1.9263 - accuracy: 0.3812
Epoch 173/1000
1/1 [==============================] - 0s 25ms/step - loss: 1.9238 - accuracy: 0.3562
Epoch

1/1 [==============================] - 0s 32ms/step - loss: 1.4722 - accuracy: 0.5625
Epoch 246/1000
1/1 [==============================] - 0s 31ms/step - loss: 1.4667 - accuracy: 0.5625
Epoch 247/1000
1/1 [==============================] - 0s 20ms/step - loss: 1.4610 - accuracy: 0.5750
Epoch 248/1000
1/1 [==============================] - 0s 22ms/step - loss: 1.4551 - accuracy: 0.5625
Epoch 249/1000
1/1 [==============================] - 0s 23ms/step - loss: 1.4491 - accuracy: 0.5875
Epoch 250/1000
1/1 [==============================] - 0s 19ms/step - loss: 1.4433 - accuracy: 0.5813
Epoch 251/1000
1/1 [==============================] - 0s 27ms/step - loss: 1.4375 - accuracy: 0.5875
Epoch 252/1000
1/1 [==============================] - 0s 22ms/step - loss: 1.4319 - accuracy: 0.6000
Epoch 253/1000
1/1 [==============================] - 0s 19ms/step - loss: 1.4264 - accuracy: 0.5938
Epoch 254/1000
1/1 [==============================] - 0s 23ms/step - loss: 1.4210 - accuracy: 0.6000
Epoch

...

1/1 [==============================] - 0s 29ms/step - loss: 0.2402 - accuracy: 0.9375
Epoch 651/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.2390 - accuracy: 0.9375
Epoch 652/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.2378 - accuracy: 0.9375
Epoch 653/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.2366 - accuracy: 0.9375
Epoch 654/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.2354 - accuracy: 0.9375
Epoch 655/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.2343 - accuracy: 0.9375
Epoch 656/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.2331 - accuracy: 0.9375
Epoch 657/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.2320 - accuracy: 0.9375
Epoch 658/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.2308 - accuracy: 0.9375
Epoch 659/1000
1/1 [==============================] - 0s 30ms/step - loss: 0.2297 - accuracy: 0.9375
Epoch

1/1 [==============================] - 0s 22ms/step - loss: 0.1720 - accuracy: 0.9375
Epoch 732/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.1714 - accuracy: 0.9375
Epoch 733/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.1709 - accuracy: 0.9375
Epoch 734/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.1704 - accuracy: 0.9375
Epoch 735/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.1698 - accuracy: 0.9375
Epoch 736/1000
1/1 [==============================] - 0s 25ms/step - loss: 0.1693 - accuracy: 0.9375
Epoch 737/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.1688 - accuracy: 0.9375
Epoch 738/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.1683 - accuracy: 0.9375
Epoch 739/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.1678 - accuracy: 0.9375
Epoch 740/1000
1/1 [==============================] - 0s 29ms/step - loss: 0.1673 - accuracy: 0.9375
Epoch

1/1 [==============================] - 0s 28ms/step - loss: 0.1411 - accuracy: 0.9375
Epoch 813/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.1408 - accuracy: 0.9375
Epoch 814/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.1406 - accuracy: 0.9375
Epoch 815/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.1403 - accuracy: 0.9375
Epoch 816/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.1401 - accuracy: 0.9375
Epoch 817/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.1399 - accuracy: 0.9375
Epoch 818/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.1396 - accuracy: 0.9375
Epoch 819/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.1394 - accuracy: 0.9375
Epoch 820/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.1391 - accuracy: 0.9375
Epoch 821/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.1389 - accuracy: 0.9375
Epoch

1/1 [==============================] - 0s 38ms/step - loss: 0.1259 - accuracy: 0.9375
Epoch 894/1000
1/1 [==============================] - 0s 19ms/step - loss: 0.1257 - accuracy: 0.9375
Epoch 895/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.1256 - accuracy: 0.9375
Epoch 896/1000
1/1 [==============================] - 0s 24ms/step - loss: 0.1254 - accuracy: 0.9375
Epoch 897/1000
1/1 [==============================] - 0s 26ms/step - loss: 0.1253 - accuracy: 0.9375
Epoch 898/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.1252 - accuracy: 0.9375
Epoch 899/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.1250 - accuracy: 0.9375
Epoch 900/1000
1/1 [==============================] - 0s 17ms/step - loss: 0.1249 - accuracy: 0.9375
Epoch 901/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.1248 - accuracy: 0.9375
Epoch 902/1000
1/1 [==============================] - 0s 23ms/step - loss: 0.1246 - accuracy: 0.9375
Epoch

1/1 [==============================] - 0s 25ms/step - loss: 0.1171 - accuracy: 0.9375
Epoch 974/1000
1/1 [==============================] - 0s 27ms/step - loss: 0.1170 - accuracy: 0.9375
Epoch 975/1000
1/1 [==============================] - 0s 22ms/step - loss: 0.1170 - accuracy: 0.9375
Epoch 976/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.1169 - accuracy: 0.9375
Epoch 977/1000
1/1 [==============================] - 0s 42ms/step - loss: 0.1168 - accuracy: 0.9375
Epoch 978/1000
1/1 [==============================] - 0s 33ms/step - loss: 0.1167 - accuracy: 0.9375
Epoch 979/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.1166 - accuracy: 0.9375
Epoch 980/1000
1/1 [==============================] - 0s 21ms/step - loss: 0.1165 - accuracy: 0.9375
Epoch 981/1000
1/1 [==============================] - 0s 28ms/step - loss: 0.1165 - accuracy: 0.9375
Epoch 982/1000
1/1 [==============================] - 0s 20ms/step - loss: 0.1164 - accuracy: 0.9375
Epoch

In [8]:
print(text)

for vec in np.array([train_X[0]]):
    for idx in vec:
        print(index_to_char[idx], end= "")

for i in range(16):
    pred = model.predict(encoded_X[i].reshape(1,10,25))
    pred = np.argmax(pred, 2)
    for vec in pred:
        for idx in vec:
            print(index_to_char[idx], end= "")

if you want to build a ship, don't drum up people together to collect wood and don't assign them tasks and work, but rather teach them to long for the endless immensity of the sea.
if you want to build a ship, don't drum n  heople together t  collect nood and don't assign  hem tas s and work, but ratker teach them to lo g for the endless iomensity o